In [ ]:
import findspark
findspark.init()

import pyspark
import random

import elastic.container
from elastic.record_event import RecordEvent

In [ ]:
@RecordEvent
def cell_2():
    sc = pyspark.SparkContext(appName="Pi")
    num_samples = 100000000
    return sc, num_samples

In [ ]:
sc, num_samples = cell_2()

In [ ]:
def inside(p):     
  x, y = random.random(), random.random()
  return x*x + y*y < 1

In [ ]:
@RecordEvent
def cell_3():
    count = sc.parallelize(range(0, num_samples)).filter(inside).count()

    pi = 4 * count / num_samples
    print(pi)

    sc.stop()
    return count, pi

In [ ]:
count, pi = cell_3()

In [ ]:
for x in elastic.container.operation_events:
    print(x, "\n")

In [ ]:
for x in elastic.container.data_events:
    print(x, "\n")

In [ ]:
parent_lookup = {oe: set() for oe in elastic.container.operation_events}
children_lookup = {oe: set() for oe in elastic.container.operation_events}

In [ ]:
for oe in elastic.container.operation_events:
    for related_data_event in elastic.container.operation_event_lookup[oe.exec_uuid].related_data_events:
        parent_oe = related_data_event.prev_operation_event
        
        parent_lookup[oe].add(parent_oe)
        children_lookup[parent_oe].add(oe)